In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.layers import Dropout
from keras import regularizers
import numpy as np
from keras.datasets import imdb
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

Using TensorFlow backend.


In [23]:
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
X_train = np.load('x_train.npz')['arr_0']
y_train = np.load('train_genres.npz')['arr_0']
#y_train = np.load('y_train.npz')['arr_0']

X_val = np.load('x_valid.npz')['arr_0']
y_val= np.load('valid_genres.npz')['arr_0']
#y_val = np.load('y_val.npz')['arr_0']
X_test = np.load('x_test.npz')['arr_0']
y_test= np.load('y_test.npz')['arr_0']

In [29]:
# truncate and pad input sequences
max_review_length = 300
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_val = sequence.pad_sequences(X_val, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)


In [13]:
y_train_binary = np.array(y_train > 6).astype(int)
y_val_binary = np.array(y_val > 6).astype(int)
y_test_binary = np.array(y_test > 6).astype(int)

In [5]:
batch_size = 64
def batch_gen():
    for i in range(X_train.shape[0]//batch_size):
        X = X_train[i*batch_size:(i+1)*batch_size]
        ls = []
        for i in X:
            ls.append(len(i))
        
        X = sequence.pad_sequences(X, maxlen=max(ls))
        #Y = y_train[i*batch_size:(i+1)*batch_size]
        yield X, y_train


In [24]:
#y_train = np.load('y_train.npz')['arr_0']
y_train_one = to_categorical(y_train)
y_valid_one = to_categorical(y_val)

In [19]:
y_train.shape
embeddings = np.load('wembeddings300.npz')['arr_0']
vocab_size = embeddings.shape[0]

In [27]:
y_train_one.shape
y_valid_one.shape

(755, 19)

In [37]:
# create the model
embedding_vecor_length = 300
model = Sequential()
#model.add(Embedding(vocab_size, embedding_vecor_length, input_length=max_review_length, embeddings_initializer=keras.initializers.Constant(value=embeddings)))
model.add(Embedding(vocab_size, 64))
model.add(LSTM(100,kernel_regularizer=regularizers.l2(0.01)))

model.add(Dense(19, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train_one, validation_data=(X_val, y_valid_one), epochs=50, batch_size=32,verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, None, 64)          3644032   
_________________________________________________________________
lstm_8 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_8 (Dense)              (None, 19)                1919      
Total params: 3,711,951
Trainable params: 3,711,951
Non-trainable params: 0
_________________________________________________________________
None
Train on 2653 samples, validate on 755 samples
Epoch 1/50
2653/2653 [==============================] - 33s 13ms/step - loss: 3.1474 - acc: 0.2182 - val_loss: 2.7105 - val_acc: 0.2278

In [81]:
# Final evaluation of the model
scores = model.evaluate(X_val, y_valid_one, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 90.00%


In [118]:

output = model.predict(X_train[1:2,:])
print(output)
print(np.argmax(output))
print(y_train[0])

[[ 0.00243329  0.00992311  0.02049176  0.02935202  0.29173535  0.00349352
   0.05166275  0.31436151  0.26104766  0.01549904]]
7
7


In [35]:

(values,counts) = np.unique(y_val,return_counts=True)
ind=np.argmax(counts)
print(ind)

16


In [36]:
float(counts[16])/np.sum(counts)

0.22781456953642384

In [159]:
values
counts
k = 7
print(float(np.sum(counts[0:k]))/np.sum(counts[0:10]))


0.552317880795


In [119]:
val_outs = model.predict(X_val)

In [128]:
preds = np.argmax(val_outs,1)
print(float(np.sum(preds == y_val)))/len(y_val)

0.218543046358


[7 6 6 8 6 8 7 8 6 6 8 6 5 8 8 7 6 6 6 7 6 4 6 6 6 4 6 4 6 8 5 7 8 8 5 6 3
 6 5 2 8 2 8 5 8 3 4 7 8 8 3 7 8 8 8 5 5 4 8 3 6 5 6 4 6 8 5 8 6 7 3 5 8 5
 6 6 3 8 3 4 8 2 6 8 5 6 5 6 6 8 6 8 5 3 7 3 4 6 8 3 6 3 6 4 4 3 8 8 5 5 8
 8 8 5 8 5 8 8 6 7 3 3 6 5 3 3 4 8 6 3 6 4 6 5 4 4 8 3 3 4 5 2 3 7 3 8 7 3
 8 8 8 8 6 3 8 6 6 3 8 5 5 6 8 8 8 6 4 5 8 4 8 8 7 3 8 3 8 7 5 6 3 3 6 8 5
 8 4 6 7 6 8 8 6 6 4 8 8 7 8 6 6 7 6 8 4 8 8 8 6 7 7 8 6 6 7 8 4 4 4 7 8 5
 7 8 8 8 7 5 8 8 5 8 8 3 8 6 8 6 8 2 6 4 7 8 6 2 6 6 6 8 6 4 6 6 7 8 8 6 7
 4 8 6 8 8 6 5 3 7 5 5 7 5 7 4 6 8 8 8 5 6 6 5 8 3 8 7 8 8 6 7 6 7 8 8 2 7
 7 3 5 2 7 5 5 5 7 6 5 7 7 4 8 6 4 7 3 6 7 6 2 5 6 4 7 8 2 4 8 8 7 8 8 8 4
 7 7 6 3 6 4 7 6 8 4 8 8 8 7 3 5 4 7 6 7 3 7 3 5 6 4 8 3 5 3 3 5 6 5 6 8 8
 8 3 7 8 3 3 6 5 4 8 8 7 7 8 4 2 5 5 4 3 8 5 2 2 6 6 8 8 8 2 3 6 6 3 6 4 8
 3 4 3 5 3 8 8 5 6 5 7 8 3 8 8 3 8 4 8 6 8 8 8 7 5 6 7 7 4 8 7 7 7 2 8 6 5
 4 8 3 8 8 4 7 6 6 4 3 6 6 4 4 6 4 6 7 6 7 4 6 8 8 7 7 5 8 4 5 8 6 4 3 7 6
 3 6 7 8 5 4 3 5 3 6 6 8 